In [1]:
import pandas
import datetime
import numpy
from scipy.optimize import curve_fit
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import ticker
%matplotlib inline
from matplotlib import rcParams

In [2]:
datFileName = "../data/processedSaps.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ')
# # add dst_bins
dstBins = [ -150, -75, -50, -25, -10, 10 ]
sapsDataDF = pandas.concat( [ sapsDataDF, \
                    pandas.cut( sapsDataDF["dst_index"], \
                               bins=dstBins ) ], axis=1 )
sapsDataDF.columns = [ "dateStr", "sapsLat", "sapsMLT", \
                      "sapsVel", "radId", "poesLat", "poesMLT", \
                      "dst_date", "dst_index", "time", "dst_bin" ]
sapsDataDF = sapsDataDF.drop(["poesLat", "poesMLT"], 1)
sapsDataDF["sapsMLTRounded"] = sapsDataDF["sapsMLT"].map(lambda x: round(x) )
sapsDataDF.head()

,dateStr,sapsLat,sapsMLT,sapsVel,radId,dst_date,dst_index,time,dst_bin,sapsMLTRounded
0,20110107,56.5,17.7543,308.2077,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
1,20110107,55.5,18.0147,224.1588,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
2,20110107,56.5,17.8749,307.4328,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
3,20110107,55.5,18.1324,222.4787,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0
4,20110107,56.5,17.9955,305.4201,33.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]",18.0


In [3]:
sapsNumRadsSer = sapsDataDF.groupby(["dateStr"]).agg( {"radId": pandas.Series.nunique} )

In [4]:
# get times, dates when the number of data points observed
# is greater than 50 points at a given time.
sapsDateTimeCount = sapsDataDF.groupby(["dateStr", "time"]).count()
sapsDateTimeCount = sapsDateTimeCount[ sapsDateTimeCount["sapsLat"] > 50 ].reset_index()
sapsDateTimeCount = sapsDateTimeCount[['dateStr','time', 'sapsLat']]
sapsDateTimeCount.columns = [ "dateStr", "time", "nPoints" ]
# Now get data points when atleast 4 radars were making observations
sapsUniqRadsCnt = sapsDataDF.groupby(["dateStr","time"]).agg( {"radId": pandas.Series.nunique} )
sapsUniqRadsCnt = sapsUniqRadsCnt[ sapsUniqRadsCnt >= 4 ].dropna().reset_index()
sapsUniqRadsCnt.columns = [ "dateStr", "time", "nRads" ]
# Merge both to get good dates where we can get velocity measurements
selVelDatesDF = pandas.merge( sapsUniqRadsCnt, \
                             sapsDateTimeCount, \
                             on=[ "dateStr", "time" ], how="inner" )
# selVelDatesDF.head()
# save as csv file
# selVelDatesDF.to_csv("../data/sapsVelDatesTimes.txt", sep=' ', index=False)

In [5]:
def convert_to_datetime(row):
    currDateStr = str( int( row["dateStr"] ) )
#     return currDateStr
    if row["time"] < 10:
        currTimeStr = "000" + str( int( row["time"] ) )
    elif row["time"] < 100:
        currTimeStr = "00" + str( int( row["time"] ) )
    elif row["time"] < 1000:
        currTimeStr = "0" + str( int( row["time"] ) )
    else:
        currTimeStr = str( int( row["time"] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

In [6]:
# Merge the selected date time DF with sapsDataDF
# to get the radar ids as well
selVelDatesDF = pandas.merge( selVelDatesDF, sapsDataDF, on=["dateStr","time"] )
selVelDatesDF = selVelDatesDF[ [ 'dateStr','time', 'radId' ] ]
selVelDatesDF = selVelDatesDF.drop_duplicates()
selVelDatesDF["dtObj"] = selVelDatesDF.apply( convert_to_datetime, axis=1 )
# selVelDatesDF.to_csv("../data/XuelingSaps.txt", sep=' ', index=False)
selVelDatesGrps = selVelDatesDF.groupby( ["dateStr", "radId"] )
selVelDatesDF.head()

,dateStr,time,radId,dtObj
0,20110205,230,206.0,20110205
20,20110205,230,207.0,20110205
29,20110205,230,205.0,20110205
36,20110205,230,33.0,20110205
98,20110205,300,206.0,20110205


In [7]:
velDateTimeRadDF = selVelDatesGrps.aggregate(lambda x: tuple(x)).reset_index()
# Get the difference between consecutive elements in the time tuple
# We can determine the time ranges for each radar this way...
velDateTimeRadDF["timeObj"] = velDateTimeRadDF["time"].map(\
                                    lambda x: numpy.ediff1d(x) )
velDateTimeRadDF.head()

,dateStr,radId,time,dtObj,timeObj
0,20110205,33.0,"(230, 300, 330)","(20110205, 20110205, 20110205)","[70, 30]"
1,20110205,204.0,"(330,)","(20110205,)",[]
2,20110205,205.0,"(230, 300, 330)","(20110205, 20110205, 20110205)","[70, 30]"
3,20110205,206.0,"(230, 300, 330)","(20110205, 20110205, 20110205)","[70, 30]"
4,20110205,207.0,"(230, 300, 330)","(20110205, 20110205, 20110205)","[70, 30]"


In [8]:
# Pivot the table so that date is row,
# radId is col and time is the value!!!
pivDF = velDateTimeRadDF.pivot(index='dateStr', columns='radId', values='time')
pivDF.head()

radId,32.0,33.0,204.0,205.0,206.0,207.0,208.0,209.0
dateStr,,,,,,,,
20110205,None,"(230, 300, 330)","(330,)","(230, 300, 330)","(230, 300, 330)","(230, 300, 330)",None,None
20110302,"(100, 200, 230)","(100, 200, 230)",None,"(200, 230)","(100, 230)","(100, 200, 230)",None,None
20110310,"(600,)","(600,)",None,"(600,)","(600,)","(600,)",None,None
20110312,"(30,)","(30,)",None,"(30,)",None,"(30,)",None,None
20110403,"(400, 430, 500, 530, 600)","(400, 430, 500, 530, 600)",None,None,"(400, 430, 500, 530, 600)","(400, 430, 500, 530, 600)",None,None


In [9]:
import datetime
print datetime.datetime.strptime( "230", "%H%M" )

1900-01-01 23:00:00
